<a href="https://colab.research.google.com/github/tarupathak30/rag-astronomy-chatbot-/blob/main/exoplanets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# create folder
folder_name = "exoplanet_data"
os.makedirs(folder_name, exist_ok=True)

In [2]:
from google.colab import files
uploaded = files.upload()  # This will open a file picker


Saving exoplanets.json to exoplanets.json


In [3]:
import shutil

for filename in uploaded.keys():
    shutil.move(filename, os.path.join(folder_name, filename))

print("Files uploaded and moved into exoplanet_data/")

Files uploaded and moved into exoplanet_data/


In [4]:
!pip install langchain

In [5]:
!pip install langchain_community faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successful

In [6]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# pure_rag.py - paste into a Colab cell
import os
import torch
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List

In [11]:
# --------- Config ----------
MODEL_ID = "google/gemma-2-2b-it"   # your local HF Gemma model id
EMBED_MODEL = "sentence-transformers/all-mpnet-base-v2"
CHUNK_SIZE = 400
CHUNK_OVERLAP = 60
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOP_K = 5
# ---------------------------

In [12]:
# 1) Load LLM (Gemma) and tokenizer
print("Loading Gemma model... (this may take a minute)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32,
    device_map="auto" if DEVICE=="cuda" else None
)
print("Gemma loaded on", DEVICE)

Loading Gemma model... (this may take a minute)


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Gemma loaded on cuda


In [13]:
# 2) Load embedding model
print("Loading embedding model...")
embed_model = SentenceTransformer(EMBED_MODEL, device=DEVICE)
embed_dim = embed_model.get_sentence_embedding_dimension()
print("Embedding dim:", embed_dim)

Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dim: 768


In [14]:
import json

def load_json_from_folder(folder: str) -> List[str]:
    texts = []
    for fname in os.listdir(folder):
        if fname.endswith(".json"):
            with open(os.path.join(folder, fname), "r", encoding="utf-8") as f:
                data = json.load(f)
                for obj in data:
                    texts.append(json.dumps(obj, ensure_ascii=False))
    return texts


In [15]:
def chunk_text(text: str, chunk_size: int=CHUNK_SIZE, overlap: int=CHUNK_OVERLAP) -> List[str]:
    tokens = text.split()
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

In [16]:
# 4) Build index from documents (call once)
def build_faiss_index_from_folder(folder: str):
    texts = load_json_from_folder(folder)
    docs = []
    for t in texts:
        docs.extend(chunk_text(t))
    print(f"Total chunks: {len(docs)}")

    # compute embeddings in batches to avoid OOM
    batch_size = 64
    embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i+batch_size]
        emb = embed_model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        embeddings.append(emb)
    import numpy as np
    embeddings = np.vstack(embeddings).astype('float32')

    # normalize (for cosine similarity via inner product)
    faiss.normalize_L2(embeddings)

    # build index
    index = faiss.IndexFlatIP(embed_dim)  # inner-product on normalized vectors = cosine
    index.add(embeddings)
    print("FAISS index built, n =", index.ntotal)

    # keep mapping
    meta = {"docs": docs, "embeddings": embeddings}
    return index, meta

In [17]:
# 5) Retrieve top-k
import numpy as np
def retrieve(query: str, index, meta, top_k: int=TOP_K):
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    hits = []
    for idx in I[0]:
        hits.append(meta["docs"][idx])
    return hits

In [61]:
PROMPT_TEMPLATE ="""
You are a reasoning engine.

The question is as follows :
{question}

Given the following JSON objects representing exoplanets:
{context}

DO THE BELOW STEPS TO ANSWER THE QUESTION:
Step 0 : Frame the question as per the query, don't make any mistake in that.
Step 1: Extract each planet's name and what is asked in the query.
Step 2: Understand the query completely.
Step 3: Explain your reasoning step-by-step to answer the query with explaining each choice of yours.

And the most important step is next one
Step 4: Output your conclusion in one sentence.

Answer:
"""



In [19]:
def build_prompt(context_chunks: List[str], question: str, max_context_chars=3000) -> str:
    # join chunks (keep it under a limit)
    joined = "\n\n".join(context_chunks)
    if len(joined) > max_context_chars:
        joined = joined[:max_context_chars]
    return PROMPT_TEMPLATE.format(context=joined, question=question)

In [53]:
# 7) Call LLM to generate (use model.generate)
def generate_answer(prompt: str, max_new_tokens=400, temperature=0.0):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(llm.device)
    with torch.no_grad():
        outputs = llm.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # sometimes the model echoes the prompt; remove prompt portion if present
    if prompt.strip() in text:
        return text.replace(prompt.strip(), "").strip()
    return text.strip()

# --------------------------
# USAGE (example)
# --------------------------
# 1) Build the index from your folder (only once)
folder = "/content/exoplanet_data"   # <-- change if needed
index, meta = build_faiss_index_from_folder(folder)

Total chunks: 6052
FAISS index built, n = 6052


In [21]:
# 2) Query function
def answer_query(question: str, top_k:int=TOP_K):
    hits = retrieve(question, index, meta, top_k=top_k)
    prompt = build_prompt(hits, question)
    ans = generate_answer(prompt)
    return {"answer": ans, "context": hits}



In [62]:
# Example run:
q = "Which exoplanet has the shortest orbital period?"
out = answer_query(q, top_k=5)
print("=== ANSWER ===")
print(out["answer"])

=== ANSWER ===
Step 0: Which exoplanet has the shortest orbital period?

Step 1: 
- Kepler-1660 AB b
- Kepler-338 e
- Kepler-1972 c
- Kepler-1931 b
- Kepler-1941 b

Step 2: The query asks for the exoplanet with the shortest orbital period. We need to find the planet with the smallest orbital period among the given exoplanets.

Step 3:  Let's analyze the orbital period of each planet:
- Kepler-1660 AB b: 239.5044 days
- Kepler-338 e: 9.341 days
- Kepler-1972 c: 11.3295 days
- Kepler-1931 b: 38.5760994 days
- Kepler-1941 b: 31.33510017 days

Based on the orbital period, Kepler-338 e has the shortest orbital period of 9.341 days.

Step 4:  Kepler-338 e has the shortest orbital period among the given exoplanets.


In [ ]:
# def retrieve(query, k=5):
#     q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
#     faiss.normalize_L2(q_emb)

#     D, I = index.search(q_emb, k)

#     print("\n=== Retrieved IDs ===")
#     print(I)
#     print("\n=== Retrieved Chunks ===")
#     for idx in I[0]:
#         print(meta["docs"][idx][:300], "...\n")

#     return [meta["docs"][i] for i in I[0]]


In [ ]:
retrieve(q)

In [ ]:
import inspect
print(inspect.signature(retrieve))
